In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,LSTM
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 256
num_samples=10000
data_path='fra.txt'

In [2]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text,target_text,_=line.split('\t')
    target_text= '\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
len(target_characters)

93

In [7]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [8]:
input_token_index = dict(
    [(char,i) for i ,char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char,i) for i ,char in enumerate(target_characters)]
)

In [9]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [10]:
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [12]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t ,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i,t+1:,input_token_index[' ']] = 1.
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]] = 1.
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]] = 1.
    decoder_input_data[i,t+1:,target_token_index[' ']] = 1.
    decoder_target_data[i,t:,target_token_index[' ']] = 1.

In [13]:
# define the input sequence
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c = encoder(encoder_inputs)
encoder_states = [state_h,state_c]

In [17]:
# setup the decoder
decoder_inputs = Input(shape=(None,num_decoder_tokens))
decoder_lstm = LSTM(latent_dim,return_state=True,return_sequences=True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
# define the model 
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 21s 152ms/step - loss: 0.0742 - accuracy: 0.7244 - val_loss: 0.0226 - val_accuracy: 0.7049
Epoch 2/100
125/125 [==============================] - 19s 148ms/step - loss: 0.0193 - accuracy: 0.7373 - val_loss: 0.0190 - val_accuracy: 0.7049
Epoch 3/100
125/125 [==============================] - 19s 149ms/step - loss: 0.0171 - accuracy: 0.7375 - val_loss: 0.0179 - val_accuracy: 0.7049
Epoch 4/100
125/125 [==============================] - 20s 157ms/step - loss: 0.0163 - accuracy: 0.7376 - val_loss: 0.0175 - val_accuracy: 0.7049
Epoch 5/100
125/125 [==============================] - 18s 148ms/step - loss: 0.0158 - accuracy: 0.7368 - val_loss: 0.0172 - val_accuracy: 0.7049
Epoch 6/100
125/125 [==============================] - 19s 155ms/step - loss: 0.0155 - accuracy: 0.7356 - val_loss: 0.0168 - val_accuracy: 0.6968
Epoch 7/100
125/125 [==============================] - 18s 148ms/step - loss: 0.0152 - accuracy: 0.7353 - val_loss: 0.0167 -

In [25]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i,char) for char,i in input_token_index.items())
reverse_target_char_index = dict((i,char) for char,i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

for seq_index in range(1):
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('input sequence : ',input_texts[seq_index])
    print('decoded sequence : ',decode_sequence)

1/1 [==============================] - 0s 19ms/step
-
input sequence :  Go.
decoded sequence :  <function decode_sequence at 0x00000119FFFD7EC0>
